In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
import warnings
warnings.filterwarnings('ignore')

# Load and prepare the dataset
df = pd.read_csv('dog_aggression_dataset.csv')

# Heart rate analysis from the dataset shows:
# Level 0 (Calm): Average 86.7 BPM
# Level 1 (Normal): Average 110.5 BPM  
# Level 2 (Stressed): Average 135.7 BPM
# Level 3 (Highly Stressed): Average 158.3 BPM
# Level 4 (Aggressive): Average 165.1 BPM

class DogBehaviorAnalyzer:
    def __init__(self):
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.scaler = StandardScaler()
        self.behavior_labels = {
            0: "Calm/Resting",
            1: "Normal Activity", 
            2: "Stressed/Anxious",
            3: "Highly Agitated",
            4: "Aggressive/Critical"
        }
        
    def extract_features(self, heart_rates):
        """Extract features from 10 heart rate readings"""
        hr_array = np.array(heart_rates)
        
        features = {
            'hr_mean': np.mean(hr_array),
            'hr_std': np.std(hr_array),
            'hr_min': np.min(hr_array),
            'hr_max': np.max(hr_array),
            'hr_range': np.max(hr_array) - np.min(hr_array),
            'hr_trend': np.polyfit(range(len(hr_array)), hr_array, 1)[0],
            'hr_peak_ratio': np.max(hr_array) / np.mean(hr_array),
            'hr_variability': np.std(hr_array) / np.mean(hr_array) if np.mean(hr_array) > 0 else 0
        }
        
        return np.array(list(features.values())).reshape(1, -1)
    
    def train_model(self, df):
        """Train the model using the dataset"""
        # Create training features from dataset
        X_features = []
        y_labels = []
        
        for _, row in df.iterrows():
            # Simulate 10 readings based on actual heart rate and HRV
            base_hr = row['heart_rate_bpm']
            hrv = row['hrv_rmssd']
            
            # Generate simulated 10-reading sequence
            np.random.seed(int(base_hr) % 1000)
            readings = np.random.normal(base_hr, hrv/3, 10)
            readings = np.clip(readings, 50, 200)  # Realistic HR bounds
            
            features = self.extract_features(readings)
            X_features.append(features[0])
            y_labels.append(row['aggression_level'])
        
        X = np.array(X_features)
        y = np.array(y_labels)
        
        # Split and train
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Train model
        self.model.fit(X_train_scaled, y_train)
        
        # Evaluate
        y_pred = self.model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        
        print(f"Model trained successfully!")
        print(f"Accuracy: {accuracy:.3f}")
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))
        
        return accuracy
    
    def analyze_behavior(self, heart_rates):
        """Analyze behavior from 10 heart rate readings"""
        if len(heart_rates) != 10:
            return "Error: Exactly 10 heart rate readings required"
        
        # Extract features
        features = self.extract_features(heart_rates)
        features_scaled = self.scaler.transform(features)
        
        # Predict
        prediction = self.model.predict(features_scaled)[0]
        probability = self.model.predict_proba(features_scaled)[0]
        
        # Get behavior label
        behavior = self.behavior_labels[prediction]
        confidence = probability[int(prediction)]
        
        # Calculate statistics
        avg_hr = np.mean(heart_rates)
        hr_trend = np.polyfit(range(len(heart_rates)), heart_rates, 1)[0]
        hr_std = np.std(heart_rates)
        
        # Generate detailed analysis notes
        analysis_notes = self.generate_analysis_notes(
            heart_rates, prediction, avg_hr, hr_trend, hr_std, confidence
        )
        
        return {
            'behavior': behavior,
            'aggression_level': int(prediction),
            'confidence': float(confidence),
            'avg_heart_rate': float(avg_hr),
            'heart_rate_trend': float(hr_trend),
            'analysis_notes': analysis_notes,
            'activate_calming': prediction >= 2,  # Activate for stress level 2+
            'alert_level': self.get_alert_level(prediction)
        }
    
    def generate_analysis_notes(self, heart_rates, prediction, avg_hr, hr_trend, hr_std, confidence):
        """Generate detailed AI analysis notes"""
        notes = []
        
        # Heart rate analysis
        if avg_hr < 90:
            notes.append(f"Average heart rate of {avg_hr:.1f} BPM indicates a relaxed state.")
        elif avg_hr < 120:
            notes.append(f"Average heart rate of {avg_hr:.1f} BPM shows normal activity levels.")
        elif avg_hr < 150:
            notes.append(f"Elevated average heart rate of {avg_hr:.1f} BPM suggests stress or anxiety.")
        else:
            notes.append(f"High average heart rate of {avg_hr:.1f} BPM indicates significant stress or potential aggression.")
        
        # Trend analysis
        if hr_trend > 3:
            notes.append("Heart rate showing strong upward trend - stress levels increasing.")
        elif hr_trend > 1:
            notes.append("Slight upward trend in heart rate - monitor for escalation.")
        elif hr_trend < -3:
            notes.append("Heart rate decreasing - dog may be calming down.")
        elif hr_trend < -1:
            notes.append("Slight downward trend - stress levels stabilizing.")
        else:
            notes.append("Heart rate remains stable over monitoring period.")
        
        # Variability analysis
        if hr_std > 20:
            notes.append("High heart rate variability suggests emotional volatility.")
        elif hr_std < 5:
            notes.append("Low heart rate variability indicates stable emotional state.")
        
        # Behavioral recommendations
        if prediction == 0:
            notes.append("Dog is in optimal calm state. Continue current environment.")
        elif prediction == 1:
            notes.append("Normal behavior detected. No intervention needed.")
        elif prediction == 2:
            notes.append("Stress detected. Consider removing stressors or calming intervention.")
        elif prediction == 3:
            notes.append("High agitation detected. Immediate calming protocol recommended.")
        else:
            notes.append("CRITICAL: Aggressive behavior likely. Activate all calming measures immediately.")
        
        # Confidence note
        notes.append(f"Analysis confidence: {confidence*100:.1f}%")
        
        return " ".join(notes)
    
    def get_alert_level(self, prediction):
        """Get alert level for dashboard"""
        alert_levels = {
            0: "Normal",
            1: "Normal", 
            2: "Warning",
            3: "High",
            4: "Critical"
        }
        return alert_levels[prediction]
    
    def save_model(self, filepath):
        """Save the trained model"""
        model_data = {
            'model': self.model,
            'scaler': self.scaler,
            'behavior_labels': self.behavior_labels
        }
        joblib.dump(model_data, filepath)
        print(f"Model saved to {filepath}")
    
    def load_model(self, filepath):
        """Load a trained model"""
        model_data = joblib.load(filepath)
        self.model = model_data['model']
        self.scaler = model_data['scaler']
        self.behavior_labels = model_data['behavior_labels']
        print(f"Model loaded from {filepath}")

# Initialize and train the analyzer
analyzer = DogBehaviorAnalyzer()
accuracy = analyzer.train_model(df)

# Test the model with sample data
print("\n" + "="*60)
print("TESTING THE MODEL")
print("="*60)

# Test cases representing different scenarios
test_scenarios = [
    {
        'name': 'Calm Dog (Sleeping)',
        'heart_rates': [68, 70, 66, 72, 69, 67, 71, 68, 70, 69],
        'expected': 'Should be classified as Calm'
    },
    {
        'name': 'Normal Activity (Playing)',
        'heart_rates': [95, 98, 102, 96, 100, 94, 99, 97, 101, 98],
        'expected': 'Should be classified as Normal Activity'
    },
    {
        'name': 'Stressed Dog (Visitor arrived)',
        'heart_rates': [125, 130, 135, 132, 138, 140, 142, 145, 148, 150],
        'expected': 'Should be classified as Stressed/Anxious'
    },
    {
        'name': 'Agitated Dog (Territorial dispute)',
        'heart_rates': [155, 160, 165, 162, 168, 170, 172, 175, 178, 180],
        'expected': 'Should be classified as Highly Agitated'
    },
    {
        'name': 'Aggressive Dog (Fight response)',
        'heart_rates': [175, 180, 185, 180, 190, 188, 192, 195, 198, 200],
        'expected': 'Should be classified as Aggressive'
    }
]

for scenario in test_scenarios:
    print(f"\nScenario: {scenario['name']}")
    print(f"Heart rates: {scenario['heart_rates']}")
    
    result = analyzer.analyze_behavior(scenario['heart_rates'])
    
    print(f"Prediction: {result['behavior']} (Level {result['aggression_level']})")
    print(f"Confidence: {result['confidence']*100:.1f}%")
    print(f"Activate Calming: {'YES' if result['activate_calming'] else 'NO'}")
    print(f"Alert Level: {result['alert_level']}")
    print(f"Analysis: {result['analysis_notes'][:150]}...")

# Save the model
analyzer.save_model('dog_behavior_model.joblib')

print(f"\n" + "="*60)
print("IoT COLLAR INTEGRATION LOGIC")
print("="*60)

# IoT Collar Integration Function
def process_iot_heart_rate_data(heart_rate_readings, analyzer):
    """
    Function to be called by IoT collar when 10 heart rate readings are collected
    This simulates the real-time processing that would happen on the ESP32 or cloud
    """
    
    # Analyze the heart rate data
    result = analyzer.analyze_behavior(heart_rate_readings)
    
    # Create response for IoT collar
    collar_response = {
        'timestamp': pd.Timestamp.now().isoformat(),
        'collar_id': 'DOG_001',  # Would be unique collar ID
        'heart_rates': heart_rate_readings,
        'behavior_analysis': result,
        'actions': {
            'activate_ultrasonic': result['activate_calming'],
            'led_color': 'green' if result['aggression_level'] <= 1 else 
                        'yellow' if result['aggression_level'] == 2 else 'red',
            'alert_dashboard': result['aggression_level'] >= 2,
            'send_notification': result['aggression_level'] >= 3
        },
        'dashboard_update': {
            'current_status': result['behavior'],
            'alert_level': result['alert_level'], 
            'confidence': result['confidence'],
            'analysis_text': result['analysis_notes'],
            'last_updated': pd.Timestamp.now().isoformat()
        }
    }
    
    return collar_response

# Simulate IoT collar sending data
print("\nSimulating IoT collar data processing:")
sample_heart_rates = [130, 135, 142, 148, 155, 160, 165, 170, 175, 180]
collar_response = process_iot_heart_rate_data(sample_heart_rates, analyzer)

print("Collar Response:")
print(f"Behavior: {collar_response['behavior_analysis']['behavior']}")
print(f"Activate Ultrasonic: {collar_response['actions']['activate_ultrasonic']}")
print(f"LED Color: {collar_response['actions']['led_color']}")
print(f"Alert Dashboard: {collar_response['actions']['alert_dashboard']}")
print(f"Analysis: {collar_response['behavior_analysis']['analysis_notes'][:100]}...")

print(f"\n" + "="*60)
print("DASHBOARD INTEGRATION")
print("="*60)
print("This model can be integrated with your React dashboard to:")
print("1. Receive real-time heart rate data from IoT collar")
print("2. Display behavior analysis and confidence scores")
print("3. Show AI-generated analysis notes")
print("4. Trigger calming sounds when stress detected")
print("5. Update alert levels and LED colors")
print("6. Log all behavior changes for historical analysis")

Model trained successfully!
Accuracy: 0.537

Classification Report:
              precision    recall  f1-score   support

         0.0       0.50      0.24      0.32        17
         1.0       0.62      0.67      0.64       150
         2.0       0.50      0.55      0.52       155
         3.0       0.45      0.37      0.41        67
         4.0       0.33      0.09      0.14        11

    accuracy                           0.54       400
   macro avg       0.48      0.38      0.41       400
weighted avg       0.53      0.54      0.53       400


TESTING THE MODEL

Scenario: Calm Dog (Sleeping)
Heart rates: [68, 70, 66, 72, 69, 67, 71, 68, 70, 69]
Prediction: Calm/Resting (Level 0)
Confidence: 57.0%
Activate Calming: NO
Alert Level: Normal
Analysis: Average heart rate of 69.0 BPM indicates a relaxed state. Heart rate remains stable over monitoring period. Low heart rate variability indicates stabl...

Scenario: Normal Activity (Playing)
Heart rates: [95, 98, 102, 96, 100, 94, 99, 